# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [1]:
ls

CONTRIBUTING.md  README.md        images/
LICENSE.md       data.sqlite      index.ipynb


In [2]:
import pandas as pd
import sqlite3

In [3]:
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

## Write an Equivalent Query using a Subquery

```SQL
select customerNumber,
       contactLastName,
       contactFirstName
       from customers
       join orders 
       using(customerNumber)
       where orderDate = '2003-01-31';
```

In [5]:
cur.execute('''SELECT customerNumber, contactLastName, contactFirstName
               FROM customers
               WHERE customerNumber IN (SELECT customerNumber
                                        FROM orders
                                        WHERE orderDate ='2003-01-31');''')

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [10]:
cur.execute('''SELECT productName, SUM(quantityOrdered)
               FROM products
               JOIN orderdetails
               USING(productCode)
               GROUP BY 1;''')

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,productName,SUM(quantityOrdered)
0,18th Century Vintage Horse Carriage,907
1,18th century schooner,1011
2,1900s Vintage Bi-Plane,940
3,1900s Vintage Tri-Plane,1009
4,1903 Ford Model A,883


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [9]:
#Your code here:
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT

In [18]:
cur.execute('''SELECT productName, SUM(DISTINCT(customerNumber)) AS individualCustomers
               FROM products
               JOIN orderdetails
               USING(productCode)
               JOIN orders
               USING(orderNumber)
               GROUP BY 1;''')

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,productName,individualCustomers
0,18th Century Vintage Horse Carriage,7655
1,18th century schooner,6234
2,1900s Vintage Bi-Plane,8964
3,1900s Vintage Tri-Plane,6079
4,1903 Ford Model A,5539


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products Which Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [21]:
cur.execute('''SELECT DISTINCT(employeeNumber), firstName, lastName, o.city, officeCode
               FROM employees e
               JOIN offices o
               USING(officeCode)
               JOIN customers c
               ON e.employeeNumber = c.salesRepEmployeeNumber
               JOIN orders
               USING(customerNumber)
               JOIN orderdetails
               USING(orderNumber)
               WHERE productCode IN (SELECT productCode
                                     FROM products
                                     JOIN orderdetails
                                     USING(productCode)
                                     JOIN orders
                                     USING(orderNumber)
                                     GROUP BY productCode
                                     HAVING COUNT(DISTINCT customerNumber) < 20);''')

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,employeeNumber,firstName,lastName,city,officeCode
0,1370,Gerard,Hernandez,Paris,4
1,1501,Larry,Bott,London,7
2,1337,Loui,Bondur,Paris,4
3,1166,Leslie,Thompson,San Francisco,1
4,1286,Foon Yue,Tseng,NYC,3


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit of Over 15K

In [32]:
cur.execute('''SELECT DISTINCT(employeeNumber), firstName, lastName, COUNT(customerNumber) AS highCreditCustomers
               FROM employees e
               JOIN customers c
               ON e.employeeNumber = c.salesRepEmployeeNumber
               GROUP BY 1
               HAVING AVG(creditLimit) > 15000
               ;''')

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,employeeNumber,firstName,lastName,highCreditCustomers
0,1165,Leslie,Jennings,6
1,1166,Leslie,Thompson,6
2,1188,Julie,Firrelli,6
3,1216,Steve,Patterson,6
4,1286,Foon Yue,Tseng,7
5,1323,George,Vanauf,8
6,1337,Loui,Bondur,6
7,1370,Gerard,Hernandez,7
8,1401,Pamela,Castillo,10
9,1501,Larry,Bott,8


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!